In [1]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib
import numpy as np

In [2]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [5]:
path = r"New_Testing_Protocol_Generalized.csv"
chem_path = r"Chemical Database.csv"

In [6]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [53]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan, unity_filter=True) # for now lets work wit this caase and go back and edit
amounts_added = CreateSamples_renewed.determine_component_amounts(concentration_df, plan)
complete_amounts_added = CreateSamples_renewed.calculate_component_amount_missing(amounts_added, plan)
complete_amounts_added_zeroed = CreateSamples_renewed.nan_amounts(complete_amounts_added)
complete_amounts_added_zeroed

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated component volumes from a component mass using dppc density of nan g/mL
You calculated component volumes from a component mass using dspepeg2000 density of nan g/mL
You calculated component volumes from a component mass using pfh density of 1.691 g/mL
You calculated component volumes from a component mass using ethanol density of 0.7893 g/mL
You calculated component volumes from a component mass using water density of 1.0 g/mL


,dppc concentration wtf,dspepeg2000 concentration wtf,pfh concentration wtf,ethanol concentration wtf,water concentration wtf,dppc amount mass g,dspepeg2000 amount mass g,pfh amount mass g,ethanol amount mass g,water amount mass g,dppc amount volume mL,dspepeg2000 amount volume mL,pfh amount volume mL,ethanol amount volume mL,water amount volume mL
0,0.000075,0.000035,0.00,0.000000,0.999890,0.00006,0.000028,0.000,0.000000,0.799912,0.0,0.0,0.000000,0.000000,0.799912
1,0.000075,0.000035,0.00,0.036842,0.963048,0.00006,0.000028,0.000,0.029474,0.770438,0.0,0.0,0.000000,0.037342,0.770438
2,0.000075,0.000035,0.00,0.073684,0.926206,0.00006,0.000028,0.000,0.058947,0.740965,0.0,0.0,0.000000,0.074683,0.740965
3,0.000075,0.000035,0.00,0.110526,0.889364,0.00006,0.000028,0.000,0.088421,0.711491,0.0,0.0,0.000000,0.112025,0.711491
4,0.000075,0.000035,0.00,0.147368,0.852522,0.00006,0.000028,0.000,0.117895,0.682017,0.0,0.0,0.000000,0.149366,0.682017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000075,0.000035,0.01,0.552632,0.437258,0.00006,0.000028,0.008,0.442105,0.349807,0.0,0.0,0.004731,0.560123,0.349807
496,0.000075,0.000035,0.01,0.589474,0.400416,0.00006,0.000028,0.008,0.471579,0.320333,0.0,0.0,0.004731,0.597465,0.320333
497,0.000075,0.000035,0.01,0.626316,0.363574,0.00006,0.000028,0.008,0.501053,0.290859,0.0,0.0,0.004731,0.634806,0.290859
498,0.000075,0.000035,0.01,0.663158,0.326732,0.00006,0.000028,0.008,0.530526,0.261386,0.0,0.0,0.004731,0.672148,0.261386


In [33]:
[float("nan")]*10

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [39]:
def isolate_common_column(df, common_string):
    cols = df.columns
    common_string_cols = [col for col in cols if common_string in col]
    final_df = df.copy()[common_string_cols]
    return final_df

def stock_dictionary(stock_names, stock_units, stock_values, stock_densities = None):
    """The stock name is required to be in the form 'solute n-solvent-stock' where the entry prior to the keyword stock are solvent
    and anything prior to that is assumed a solute. Will return a dictionary of the solvent and solute while pulling information from """
    if stock_densities == None:
        stock_densities = len(stock_names)*[float('nan')]
    
    stock_dict = {}
    for i, stock_name in enumerate(stock_names):
        stock_unit = stock_units[i]
        stock_value = stock_values[i]
        stock_density = stock_densities[i]
        stock_components = stock_name.split('-')
        stock_solutes = stock_components[:-2] # will always be a list
        stock_solvent = stock_components[-2]
        stock_dict[stock_name] = {'solutes': stock_solutes, 'solvents':stock_solvent, 'unit': stock_unit, 'value': stock_value, 'Density (g/mL)': stock_density}
        
    return stock_dict


def identify_common_solvents(stocks_dict):
    solvents = [stock_info['solvents'] for stock_name, stock_info in stock_dict.items()]
    common_solvents = list(set([x for x in solvents if solvents.count(x) > 1]))
    for stock_name, stock_info in stock_dict.items(): # make into identify_stocks_solvent_solutes()
        solutes = stock_info['solutes']
        solvents = stock_info['solvents']
        
        for common_solvent in common_solvents:
            if len(solutes) == 0 and common_solvent==solvents:
                stock_dict[stock_name]['Common Solvent'] = 'Pure'
            elif len(solutes) > 0 and common_solvent == solvents:
                stock_dict[stock_name]['Common Solvent'] = 'Mixture'
            else:     
                stock_dict[stock_name]['Common Solvent'] = 'None'
    return stock_dict

In [63]:
# Assumption that stocks are ordered in the same order of the concentration dataframe. 
# Each stock only contains one solvent and one solute (for the purposes of this function)
# Common solvents can be anywhere in this. 

# idea: maybe should follow similar dictionary layout of the opentrons commands dictionary as in 
#{component:{component_name:_, concentrations:[], concentration_units:wtf, density, stock:{stock_name: , stock_unit:, stock_conc:}}
# and then like this you are iterating and encapsultnig everything the only issue it makes stuff independent?  nah just calculture them all and the subtract common if needed
stock_names = plan['Stock Names']
stock_units = plan['Stock Concentration Units']
stock_concentrations = plan['Stock Concentrations']
stock_densities = plan['Stock Density (g/mL) (only for wtf)']
stock_dict = stock_dictionary(stock_names, stock_units, stock_concentrations, stock_densities)
component_dict = plan['Chemical Database']

stock_dict = identify_common_solvents(stock_dict)

complete_component_df = complete_amounts_added_zeroed
# total mass unit = 'g' then if not volume
masses = isolate_common_column(complete_component_df, 'mass')
volumes = isolate_common_column(complete_component_df, 'volume')
assert len(masses.columns) == len(stock_names)
for i, col in enumerate(masses): # iterating to match stock to component
    stock_name = stock_names[i] # the biggest issue is there no way to call and match stock and componetns
    stock_unit = stock_units[i]
    stock_concentration = stock_concentrations[i]
    stock_info = stock_dict[stock_name]
    common_solvent = stock_info['Common Solvent']
    stock_density = stock_info['Density (g/mL)']
    
    component_mass = masses[col]
    component_name = CreateSamples_renewed.identify_component_name(col)
    component_unit = CreateSamples_renewed.identify_unit(col)
    component_mw = component_dict[component_name]['Molecular Weight (g/mol)']

    stock_volumes = calculate_stock_volumes_mass_units(component_mass, component_unit, stock_concentration, stock_unit, stock_density, component_mw)
    
    complete_component_df[stock_name + ' amount volume mL'] = stock_volumes

pfh
pfh
pfh


In [73]:
stock_volumes = isolate_common_column(complete_component_df, 'stock')
stock_volumes['Total Volume'] = stock_volumes.sum(axis=1)
stock_names = [CreateSamples_renewed.identify_component_name(col) for col in stock_volumes if 'stock' in col]

# have the stock dict previously made for the common solvent
# iterate through the columns names 
# from the column name grab the stock name 
# and then call the stock info which will allow you to now grpab the common solvent determinante
# first goal: find the solo common solvent
# ok once idenified tkae it out of the stocks list along with any NONE 
# and make it into a common variable on (need to think about how it will work with many common solvents)
# then wit the remaining start to subtract based on the solvent idensidity using name from the line above.
# this should work with multiple solvents since you are calling it by solvent name and looking the solo common solvents list which ehader and referencing that one. 

['dppc-ethanol-stock',
 'dspepeg2000-ethanol-stock',
 'pfh-ethanol-stock',
 'ethanol-stock',
 'water-stock']

In [ ]:
# in the future a function will be placed before if statements that call for mass and volume units and convert them to a stnadard one like mL or g 

In [62]:
def calculate_stock_volumes_mass_units(component_mass, component_unit, stock_concentration, stock_unit, stock_density = None, component_mw = None):
    if component_unit == 'g' and stock_unit == 'wtf': # we need to specify g as it is the only thing working
        print('pfh')
        stock_mass = component_mass/stock_concentration # in g
        stock_volume = stock_mass/stock_density
    elif component_unit == 'g' and stock_unit == 'molarity':
         stock_volume = 1000*component_mass/(stock_concentration*component_mw) # in mL
    elif component_unit == 'g' and stock_unit == 'mgpermL':
         stock_volume = component_mass/(stock_concentration/1000)
    else:
        raise AssertionError("Units provided are not currently supported")
    return stock_volume

In [42]:
# this can be used from an export 
def calculate_stock_volumes_vol_units(component_volume, component_unit, stock_concentration, stock_unit, stock_density = None, stock_mw = None):
    
    # mL could really be anything
    if component_unit == 'mL' and stock_unit == 'volf': # we need to specify g as it is the only thing working
        stock_volume = component_volume/stock_concentration
    return stock_volume